In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, when, col
from pyspark.sql.functions import count as _count
from pyspark.sql.functions import min as _min
from datetime import datetime
import psutil
import multiprocessing
import findspark

In [2]:
total_memory = psutil.virtual_memory().total / (1024**3)
memory_alocated = round(total_memory / 2)
num_cores = multiprocessing.cpu_count()
print(f'{total_memory} disponível. {memory_alocated} será alocada nesse processo. {num_cores} núcleos do processador serão envolvidos no processo')
spark = ""

findspark.init()
spark = SparkSession.builder.master("local[*]") \
    .appName("source_archive") \
    .config("spark.executor.memory", f"{memory_alocated}g") \
    .config("spark.executor.instances", f"{num_cores}") \
    .getOrCreate()

7.778934478759766 disponível. 4 será alocada nesse processo. 8 núcleos do processador serão envolvidos no processo


In [3]:
class ValidaColunas:
    @staticmethod
    def create_dicit(fornecedor, df_variaveis):
    # Define um método estático chamado 'cria_dicionario' que recebe o fornecedor e um dataframe df_variaveis como entrada.
        dict_cols = {}
        # Cria um dicionário vazio chamado 'dict_cols' para armazenar as informações das colunas.
        df = df_variaveis.loc[df_variaveis['fornecedor'] == fornecedor]
        # Filtra o dataframe df_variaveis com base na coluna 'fornecedor' e armazena o resultado no dataframe df.

        df = df.sort_values(by='posicao', ascending=True)
        # Ordena o dataframe pela coluna 'posição'

        lista_colunas = df['nome_coluna'].tolist()
        # Converte a coluna 'nome_coluna' do dataframe df em uma lista e armazena-a em 'lista_colunas'.

        for col in lista_colunas:
        # Itera sobre cada coluna presente em 'lista_colunas'.
            col_info ={
            # Para cada coluna, cria um dicionário 'col_info' contendo informações relevantes sobre a coluna, e atribui ao dicionário o valor da coluna 'coluna' correspondente à coluna atual (col) do dataframe.
                    'coluna_renomeada': df['coluna_renomeada'].loc[df['nome_coluna'] == col].iloc[0],
                    # Cria uma chave no dicionário chamado 'coluna_renomeada'. df['coluna_renomeada']: Acessa a coluna 'coluna_renomeada' no dataframe 'df'.
                    # .loc[df['nome_coluna'] == col]: Filtra as linhas onde o valor da coluna 'nome_coluna' é igual ao valor de 'col'.
                    # .iloc[0]: Seleciona o primeiro valor resultante do filtro, uma vez que pode haver apenas uma correspondência. 

                    'posicao': df['posicao'].loc[df['nome_coluna'] == col].iloc[0],
                    'formato_dado_spark_origem': df['formato_dado_spark_origem'].loc[df['nome_coluna'] == col].iloc[0],
                    'formato_dado_spark_conversao': df['formato_dado_spark_conversao'].loc[df['nome_coluna'] == col].iloc[0],
                    'not_null': df['not_null'].loc[df['nome_coluna'] == col].iloc[0],
                    'standard_value': df['standard_value'].loc[df['nome_coluna'] == col].iloc[0],
                    'inicio_vigencia': pd.to_datetime(df['inicio_vigencia'].loc[df['nome_coluna'] == col].iloc[0]).strftime('%d/%m/%Y'),
                    'fim_vigencia': pd.to_datetime(df['fim_vigencia'].loc[df['nome_coluna'] == col].iloc[0]).strftime('%d/%m/%Y')
                }
            
            dict_cols[col] = col_info
            # Adiciona a entrada do dicionário 'col_info' no dicionário 'dict_cols' com a chave sendo o nome da coluna atual.

        return dict_cols
        # Retorna o dicionário com as informações das colunas.
    @staticmethod
    def valid_headers(fornecedor, sep):
        root_path = r"G:\.shortcut-targets-by-id\1kkcO2EWqo7VqwAVtJEGVmYuTF858en0s\HDI_DR\Arquivos_Source_Individuais\2023"

        dicionario_pronto = ValidaColunas.create_dicit(fornecedor, df_variaveis)
        # Comparar as colunas do df com as chaves do dicionário pronto.

        files_list = os.listdir(root_path)
        for file in files_list:
            if file.endswith('.csv'):
                csv_path = os.path.join(root_path, file)
                print(f'Caminho: {csv_path} \n')
                df = spark.read.csv(csv_path, sep=sep, encoding='utf-8', header=True)
     
            elif file.endswith('.parquet'):
                parquet_path = os.path.join(root_path, file)
                print(f'Caminho: {parquet_path} \n')
                df = spark.read.parquet(parquet_path)

                colunas_df = df.columns

                chaves_dict = dicionario_pronto.keys()

                #colunas_ausentes = set(chaves_dict) - set(colunas_df)
                # Validar nomes e ordem das colunas
                for coluna, chave in  zip(colunas_df, chaves_dict):
                    if coluna == chave:
                        print(f'A coluna {coluna} foi aprovada na validação de hearders.')



                # pares_dict = zip(colunas_df, chaves_dict)
                # conjunto_chaves_dict = set(pares_dict)
                # pares_colunas_df = zip(colunas_df, chaves_dict)
                # conjunto_colunas_df = set(pares_colunas_df)
                # colunas_ausentes = conjunto_chaves_dict - conjunto_colunas_df

                # Fazer um loop com zip...
                # Incluir diferenças dos dois lados para ver se vai pegar. (talvez precise fazer uma diferença simétrica).
                # Fazer a leitura da data do arquivo (min e max)para considerar a vigência correta. Fazer um range de cotações de toda base.
                # Desenvolver dentro do código oficial.

                if colunas_ausentes:
                    return colunas_ausentes
                else:
                    return "Todas as chaves do dicionário estão presentes como colunas no DataFrame."
                
validacao = ValidaColunas()

path = r"C:\Users\Guilherme\Documents\Talent Pool\Tropa BI\HDI\Multicalculo_validação\Dicionário de dados\DICIONARIO DE DADOS.xlsx"   

df_variaveis = pd.read_excel(path, sheet_name=1)

dicionario_pronto = validacao.create_dicit('Infocap', df_variaveis)

dicionario_pronto                
                
ValidaColunas.valid_headers('Infocap', ',')

search = ValidaColunas.valid_headers('Infocap', ',')
print(search)


None


In [6]:
validacao = ValidaColunas()

path = r"C:\Users\Guilherme\Documents\Dicionário de dados\DICIONARIO DE DADOS.xlsx"   

df_variaveis = pd.read_excel(path, sheet_name=1)

dicionario_pronto = validacao.create_dicit('Agger', df_variaveis)

dicionario_pronto


{'grupo_calculo': {'coluna_renomeada': 'cotacao_id',
  'posicao': 1,
  'formato_dado_spark_origem': 'string',
  'formato_dado_spark_conversao': 'string',
  'not_null': True,
  'standard_value': 0.0,
  'inicio_vigencia': '01/01/2023',
  'fim_vigencia': '31/12/2023'},
 'ranking': {'coluna_renomeada': 'cotacao_ranking',
  'posicao': 2,
  'formato_dado_spark_origem': 'bigint',
  'formato_dado_spark_conversao': 'int',
  'not_null': False,
  'standard_value': nan,
  'inicio_vigencia': '01/01/2023',
  'fim_vigencia': '31/12/2023'},
 'referencia': {'coluna_renomeada': 'seguradora_referencia',
  'posicao': 3,
  'formato_dado_spark_origem': 'bigint',
  'formato_dado_spark_conversao': 'int',
  'not_null': nan,
  'standard_value': nan,
  'inicio_vigencia': '01/01/2023',
  'fim_vigencia': '31/12/2023'},
 'contrato_seguradora': {'coluna_renomeada': 'seguradora_contrato',
  'posicao': 4,
  'formato_dado_spark_origem': 'double',
  'formato_dado_spark_conversao': 'int',
  'not_null': nan,
  'standard_v

In [7]:
search = ValidaColunas.valid_headers('Infocap', ',')
print(search)

None
